## Introduction <a name="introduction"></a>

Association football

Competions between national teams culminate in the quadrenail Worldcup.  FIFA rankings are used to seed teams in the tournament.  Because of the low scoring nature of football, there 




1. [Introduction](#introduction)
2. [Data](#Data)
    1. [DataFrame from File](#Datasub1)
    2. [Prepare DataFrame for Analysis](#Datasub2)
3. [Ranking Systems](#Rank)
    1. [ELO](#RankSub1)
    2. [Fifa ELO System](#RankSub2)
    3. [Regress to Mean after WorldCup ](#RankSub3)
    4. [Modify G Coefficient](#RankSub4)
    

## Data <a name="Data"></a>
The data comes from the Kaggle  as a csv file. More info about data



## DataFrame from File <a name="Datasub1"></a>
With no parsing except for defining the date column as a date object, this is what I get. For my analysis I am not going to work with the entire data set.  Here I can reduce the data set by setting a start and end date.

In [ ]:
def conv_tournament(S):
    L = set(S.unique())
    A = {'FIFA World Cup'}
    B = {'AFC Asian Cup', 'African Cup of Nations', 'Copa América', 'Gold Cup', 'Nations Cup',
         'Copa América', 'UEFA Euro'}
    C = {'FIFA World Cup qualification', 'AFC Asian Cup qualification', 'African Cup of Nations qualification',
        'Copa América qualification', 'Gold Cup qualification', 'CONCACAF Championship', 'UEFA Euro qualification'}
    E = {'Friendly'}
    D = L - A - B - C - E
    return  {60:A, 50:B, 40:C, 30:D, 20:E}

In [ ]:
def set_k(S, dict):
    A = []
    for s in S:
        for key, val in dict.items():
            if s in val:
                A.append(key)
    return A

In [63]:
def result(gdf):
    if not gdf:         return 0.50
    if gdf == abs(gdf): return 1.00
    else:               return 0.00

In [ ]:
def conv_tournament(S):
    L = set(S.unique())
    A = {'FIFA World Cup'}
    B = {'AFC Asian Cup', 'African Cup of Nations', 'Copa América', 'Gold Cup', 'Nations Cup',
         'Copa América', 'UEFA Euro'}
    C = {'FIFA World Cup qualification', 'AFC Asian Cup qualification', 'African Cup of Nations qualification',
        'Copa América qualification', 'Gold Cup qualification', 'CONCACAF Championship', 'UEFA Euro qualification'}
    E = {'Friendly'}
    D = L - A - B - C - E
    return  {60:A, 50:B, 40:C, 30:D, 20:E}

def set_k(S, dict):
    A = []
    for s in S:
        for key, val in dict.items():
            if s in val:
                A.append(key)
    return A
def result(gdf):
    if not gdf:         return 0.50
    if gdf == abs(gdf): return 1.00
    else:               return 0.00

In [1]:
# this function prepares intial data frame 
def results_2(df):   
    df.columns = ['date', 'home', 'away', 'home_score', 'away_score', 'tournament', 'city', 'country', 'neutral']  
    df['gdf']      = np.subtract(df.home_score, df.away_score)
    df['draw']     = np.where(df['gdf'] == 0, True, False)
    df['win']      = np.where(df.gdf > 0, True, False)
    df['k']        = set_k(df.tournament, conv_tournament(df.tournament)) 
    df['result']   = df.gdf.apply(lambda x : result(x))  
    df['gdf']      = df.gdf.abs()
    df = df.drop(['home_score', 'away_score', 'city', 'country', 'tournament'], axis=1)    
    return df

In [ ]:
df = results_2(df)
df.head(9)

In [ ]:
This last round of processing creates three objects:
1. dictionary with all countries ratings at end of time period
2. results DataFrame to graph rating over time
3. list of probabilities (win_expect) for winning team for analysis and figures.

## Ranking Systems <a name="Rank"></a>


ovarall discussiom

## ELO Rating System <a name="RankSub1"></a>

The World Football Elo Ratings are based on the Elo rating system, developed by Dr. Arpad Elo. This system is used by FIDE, the international chess federation, to rate chess players.

Eloratings.net applies the Elo rating system to international football, by adding a weighting for the kind of match, an adjustment for the home team advantage, and an adjustment for goal difference in the match result.

The ratings take into account all international "A" matches for which results could be found. International football data is primarily obtained from rsssf.com, theroonba.com, and soccer-db.info.

Ratings tend to converge on a team's true strength relative to its competitors after about 30 matches. Ratings for teams with fewer than 30 matches should be considered provisional.

The ratings are based on the following formulas:
**Rn = Ro + K × G x (W - We)**

Rn is the new rating, Ro is the old (pre-match) rating.

**K** is the weight constant for the tournament played:
```
60 for World Cup finals;
50 for continental championship finals and major intercontinental tournaments;
40 for World Cup and continental qualifiers and major tournaments;
30 for all other tournaments;
20 for friendly matches.
```
**G** is a constant based on the goal differential (gdf).

```
+1                 G = 1.00
+2                 G = 1.50
+3                 G = 1.75
+4 or higher       G = (11 + gdf) / 8
```
**W** is the result of the game.
```
1 for a win
0.5 for a draw
0 for a loss
```
**We** is the expected result (win expectancy). The value *dfr* is the difference in ratings plus 100 points for a team playing at home.
```
We = 1 / (10(-dfr/400) + 1)
```




In [ ]:
# this function calculates G value 
def calc_g(gdf):    # gdf is int >= 0
    if not gdf:  return 1.00
    if gdf == 1: return 1.00
    if gdf == 2: return  1.50 
    else:        return (11 + gdf) / 8  


def win_exp(dfr):
    return 1 / ((10) ** (-dfr / 400) + 1)

#creates ratings dict to begin an analysis
def start_results(date,dict):
    A = []
    for key, val in dict.items():
        S = pd.Series([date, key, int(val)])
        A.append(S)
    df = pd.DataFrame(A)
    df.columns = ['date', 'country', 'rating']
    return df

def teams_rate_begin(df):
    all = set(df.home.unique()) | set(df.away.unique())
    rtgs = {}
    for team in all:
        rtgs[team] = 1500   
    return rtgs

In [ ]:
rtgs = teams_rate_begin(df)
results = start_results(begin,rtgs)
win_probs =[]

In [ ]:
for (i, S) in df.iterrows():
   
    date = S[0]                                         #date
    home, away = S[1], S[2]                             # strings
    nuetral, draw, win = S[3], S[5], S[6]               # bools
    gdf, k =  S[4], S[7]                                # ints 
    result = S[8]                                       # float
    
    home_rtg, away_rtg = rtgs[home], rtgs[away]
    
    if not nuetral:   dfr = home_rtg + 100 - away_rtg                    # not nuetral? ==> home rating + 100                                    
    else:    dfr = home_rtg - away_rtg   
        
    expect = win_exp(dfr)
    adjust = int(np.round(calc_g(gdf) * k * (result - expect), decimals=0))  
    home_rtg += adjust
    away_rtg -= adjust
    
    if not draw:
        if win: win_probs.append(1 - expect)                            # appends list with winning team's P(expect)
        else:   win_probs.append(- expect)
            
    results = results.append(pd.DataFrame(pd.Series({'date':date, 'country':home, 'rating':home_rtg})).T) 
    results = results.append(pd.DataFrame(pd.Series({'date':date, 'country':away, 'rating':away_rtg})).T) 
    
    
    rtgs[home] = home_rtg; rtgs[away] = away_rtg 
    
results.index = range(len(results))

## Fifa ELO System <a name="RankSub2"></a>

After the 2018 world cup, FIFA adopted a modified version of the ELO system .... 


## Regress to Mean after WorldCup   <a name="RankSub3"></a>

One complaint

In [ ]:
def fifa_conv_tournament(S):
    L = set(S.unique())
    A = {'FIFA World Cup'}
    B = {'AFC Asian Cup', 'African Cup of Nations', 'Copa América', 'Gold Cup', 'Nations Cup',
         'Copa América', 'UEFA Euro'}
    C = {'FIFA World Cup qualification', 'AFC Asian Cup qualification', 'African Cup of Nations qualification',
        'Copa América qualification', 'Gold Cup qualification', 'CONCACAF Championship', 'UEFA Euro qualification'}
    E = {'Friendly'}
    D = L - A - B - C - E
    return  {60:A, 50:B, 40:C, 30:D, 20:E}

In [ ]:
def fifa_win_exp(dfr):
    return 1 / ((10) ** (-dfr / 400) + 1)

In [ ]:
def Regress_mean(dict):
    
    for key, val in dict.items():
        dict[key] = dict[key] + (1500 - dict[key]) // 2

    return dict

    

In [ ]:

test = Regress_mean(rtgs)


## Modify G Coefficient <a name="RankSub4"></a>

My belief is that teams should not .....

In [ ]:
# this function calculates G value 
def calc_g(gdf):    # gdf is int >= 0
    if gdf == 1: return 1.00
    else:        return 2.00 